In [1]:
import pandas as pd


In [3]:
pd.__version__

'2.2.3'

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet

--2025-01-16 21:13:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.197, 3.164.82.40, 3.164.82.160, ...
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.197|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 49961641 (48M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-01.parquet’

yellow_tripdata_202 100%[===================>]  47.65M  21.3MB/s    in 2.2s    

2025-01-16 21:13:29 (21.3 MB/s) - ‘yellow_tripdata_2024-01.parquet’ saved [49961641/49961641]

